In [1]:
import os
import sys
import numpy as np
import pandas as pd
from scipy.interpolate import interp1d
import matplotlib.pyplot as plt

import jax
from jax import numpy as jnp
from jax.config import config
from flax.training import checkpoints
import flax.linen as nn

import nest_asyncio
nest_asyncio.apply()

from scipy.interpolate import LinearNDInterpolator
from scipy.interpolate import NearestNDInterpolator
from scipy.interpolate import interp1d

from scipy.optimize import minimize
from scipy.stats import qmc
# Constants
from scipy.constants import Boltzmann, Avogadro
kb = Boltzmann # [J/K] Boltzman's constant
Na = Avogadro  # [mol-1] Avogadro's Number
R = Na * kb    # [J mol-1 K-1] Ideal gas constant

sys.path.append("../../")
from python_helpers.feanneos import HelmholtzModel
from python_helpers.feanneos import helper_solver_funs, helper_jitted_funs
from python_helpers.transport_properties import TransportModel_PVT_Tinv
from python_helpers import helper_get_alpha
from python_helpers import linear_activation
from python_helpers.data_figures import mie_params_of_vle_visc as mie_params_of


PRECISSION = 'float64'
if PRECISSION == 'float64':
    config.update("jax_enable_x64", True)
    type_np = np.float64
    type_jax = jnp.float64
else:
    config.update("jax_enable_x32", True)
    type_np = np.float32
    type_jax = jnp.float32

np.seterr(all="ignore")

{'divide': 'ignore', 'over': 'ignore', 'under': 'ignore', 'invalid': 'ignore'}

In [2]:
#######################################
# type and optimization configuration #
#######################################

of_type = "vle_visc"
# ranges for sigma, eps, lambda_r
l_bounds = [2.5, 100., 9.]
u_bounds = [3.5, 300., 26.]
m_base = 5
params_file = f'optimized_mie_params_{of_type}.csv'

In [3]:
filename = '../../computed_files/phase_equilibria_fluid.xlsx'
excel_file = pd.ExcelFile(filename)

df_info = pd.read_excel(excel_file, sheet_name='info')
df_vle = pd.read_excel(excel_file, sheet_name='vle')
# critical point information interpolation
input_crit_interp = df_info['alpha'].to_numpy()
output_crit_interp = df_info[['rhocad_model', 'Tcad_model', 'Pcad_model']].to_numpy()
crit_interp = interp1d(input_crit_interp, output_crit_interp.T, fill_value='extrapolate')

# Interpolating VLE
input_vle_interp = df_vle[['alpha', 'Tr_vle_model']].to_numpy()
output_vle_interp = df_vle[['P_vle_model', 'rhov_vle_model', 'rhol_vle_model']].to_numpy()
vle_interp = LinearNDInterpolator(input_vle_interp, output_vle_interp)

interpd_dict = {'crit_interp': crit_interp, 'vle_interp': vle_interp}

In [4]:
######################
# Loading FE-ANN EoS #
######################

ckpt_folder = '../../ann_models/feann_eos'

prefix_params = 'FE-ANN-EoS-params_'

###
Tscale = 'Tinv'
seed = 1
factor = 0.05
EPOCHS = 20000
traind_model_folder = f'models_{Tscale}_factor{factor:.2f}_seed{seed}'
ckpt_folder_model = os.path.join(ckpt_folder, traind_model_folder)
ckpt_Tinv = checkpoints.restore_checkpoint(ckpt_dir=ckpt_folder_model, target=None, prefix=prefix_params)
helmholtz_features = list(ckpt_Tinv['features'].values())
helmholtz_model = HelmholtzModel(features=helmholtz_features)
helmholtz_params = {'params': ckpt_Tinv['params']}
fun_dic_fluid = helper_jitted_funs(helmholtz_model, helmholtz_params)

#########################
# Shear visocsity model #
#########################

activation_dicts = {'linear': linear_activation, 'softplus': nn.softplus}
folder_visc = '../../ann_models/visc_models'
hidden_layers = 2
neurons = 30
prefix = 'logvisc-rho-Tinv-penalty'
ckpt_dir = folder_visc
seed = 0
features = hidden_layers * [neurons]
activation = 'linear'
params_prefix = f'{prefix}-seed{seed}-params_'
state_restored = checkpoints.restore_checkpoint(ckpt_dir=ckpt_dir, target=None, prefix=params_prefix)
params_logvisc = {'params': state_restored['params']}
logvisc_model = TransportModel_PVT_Tinv(features=features, output_activation=activation_dicts[activation])

logvisc_model_jit = jax.jit(lambda alpha, rhoad, Tad: logvisc_model.apply(params_logvisc, jnp.atleast_1d(alpha), jnp.atleast_1d(rhoad), jnp.atleast_1d(Tad)))
visc_fun = lambda alpha, rhoad, Tad: jnp.exp(logvisc_model_jit(alpha, rhoad, Tad))

In [5]:
# Reading data from NIST
filename = 'xenon.xlsx'
DataFile = pd.ExcelFile(filename)

if not os.path.exists(params_file):
    sampler = qmc.Sobol(d=3, scramble=False)
    sample = sampler.random_base2(m=m_base)

    scaled_sample = qmc.scale(sample, l_bounds, u_bounds)

    sigma_guess = scaled_sample[:, 0]
    epsilon_guess = scaled_sample[:, 1]
    lr_guess = scaled_sample[:, 2]

    n = len(sigma_guess)

    index = np.arange(n)
    solved = np.zeros(n, dtype=bool)
    sigma_sol = np.nan * np.ones(n)
    epsilon_sol = np.nan * np.ones(n)
    lr_sol = np.nan * np.ones(n)
    of = np.nan * np.ones(n)
    solved_success = np.zeros(n, dtype=bool)

    df = pd.DataFrame({'index': index, 
                    'sigma_guess': sigma_guess, 'epsilon_guess': epsilon_guess, 'lr_guess': lr_guess,
                    'optimized': solved, 
                    'sigma_sol': sigma_sol, 'epsilon_sol': epsilon_sol, 'lr_sol': lr_sol, 'of': of, 'solved_success': solved_success})
    
    df.to_csv(params_file, index=False)




In [6]:
# bounds to control sigma, epsilon and lr
bounds = ((None, None), (None, None), (7., 34.))

kwargs = {'DataFile': DataFile,
          'fun_dic': fun_dic_fluid,
          'visc_fun': visc_fun,
          'interpd_dict': interpd_dict,
          'lambda_a': 6.,
          'weight_rhov_vle':0.0,
          'weight_hvap':1.0,
          'add_critical_point': True,
          'weight_visc': 1.0,   
          }
args = tuple(kwargs.values())

In [7]:
df = pd.read_csv(params_file)
n = len(df)
for i in range(n):
    if not df.loc[i, 'optimized']:
        inc0 = df.loc[i, ['sigma_guess', 'epsilon_guess', 'lr_guess']].to_list()
        sol = minimize(mie_params_of, inc0, method='Nelder-Mead', args=args, bounds=bounds, options={'maxiter':10})

        df.loc[i, 'solved_success'] = sol.success
        df.loc[i, 'sigma_sol'] = sol.x[0]
        df.loc[i, 'epsilon_sol'] = sol.x[1]
        df.loc[i, 'lr_sol'] = sol.x[2]
        df.loc[i, 'of'] = sol.fun
        df.loc[i, 'optimized'] = True

        df.to_csv(params_file, index=False)
    print(i, df.loc[i, 'optimized'], df.loc[i, 'of'])

0 True 3.4596398930228305
1 True 0.1018489442396174
2 True 0.0979229149231436
3 True 0.983135628525658
4 True 164.98615152406606
5 True 0.0047400488088794
6 True 3.4996328655829725
7 True 1.582450643224779
8 True 3.443048675003239
9 True 0.0047400488089021
10 True 3.5597320864252713
11 True 0.1185535948917853
12 True 3.5156434053517907
13 True 0.0503240090065001
14 True 0.1752835678832728
15 True 0.1551743767752287
16 True 2.245209716023519
17 True 0.123047835874553
18 True 1.4577953369182757
19 True 0.004740048808866
20 True 3.512915186421726
21 True 0.0047400488089969
22 True 5.039506114974854
23 True 0.4769664568623382
24 True 3.448303387857857
25 True 0.0047400488090751
26 True 3.000293542149017
27 True 0.1426010903382515
28 True 1.0310274471892291
29 True 0.0593930925523898
30 True 3.4919437970203786
31 True 3.888025260306165


In [8]:
df = pd.read_csv(params_file)
n = len(df)

of_order = np.argsort(df['of'].to_numpy())

for i in of_order[:5]:
    if not df.loc[i, 'solved_success']:
        inc0 = df.loc[i, ['sigma_sol', 'epsilon_sol', 'lr_sol']].to_list()

        sol = minimize(mie_params_of, inc0, method='Nelder-Mead', args=args, bounds=bounds)

        df.loc[i, 'solved_success'] = sol.success
        df.loc[i, 'sigma_sol'] = sol.x[0]
        df.loc[i, 'epsilon_sol'] = sol.x[1]
        df.loc[i, 'lr_sol'] = sol.x[2]
        df.loc[i, 'of'] = sol.fun
        df.loc[i, 'optimized'] = True

        df.to_csv(params_file, index=False)


In [9]:
of_order = np.argsort(df['of'].to_numpy())
df.loc[of_order].head(n=10)

,index,sigma_guess,epsilon_guess,lr_guess,optimized,sigma_sol,epsilon_sol,lr_sol,of,solved_success
19,19,2.84375,243.75,21.21875,True,4.009259,238.493769,13.278582,0.004740,True
5,5,3.37500,275.00,11.12500,True,4.009260,238.493817,13.278587,0.004740,True
9,9,3.18750,262.50,16.43750,True,4.009260,238.493816,13.278586,0.004740,True
21,21,3.46875,218.75,14.84375,True,4.009259,238.493769,13.278582,0.004740,True
25,25,3.15625,231.25,9.53125,True,4.009260,238.493809,13.278587,0.004740,True
13,13,3.31250,237.50,22.81250,True,4.009079,264.719650,17.358153,0.050324,False
29,29,3.28125,256.25,20.15625,True,3.982060,268.535237,17.684414,0.059393,False
2,2,3.25000,150.00,13.25000,True,3.924074,190.833333,8.735185,0.097923,False
1,1,3.00000,200.00,17.50000,True,3.766667,237.777778,12.638889,0.101849,False
11,11,2.93750,212.50,12.18750,True,3.699074,217.353395,10.983796,0.118554,False


In [10]:
param_best = df.loc[of_order[0], ['sigma_sol', 'epsilon_sol', 'lr_sol']].to_list()
param_best

[4.0092594390390985, 238.4937687137886, 13.27858187487336]